In [1]:
import pandas as pd
import pickle
import numpy as np
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import random
from Class_Balancing import *

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
#  Location of file to convert
dir = "llama_generated_final.csv"


movies = pd.read_csv(dir).text
movies

0       The Hunger Games: Catching Fire is a thought-...
1        "12 Angry Men" is a gripping, thought-provok...
2        The Amazing Spider-Man 2 (2014) is a thrilli...
3       The Green Hornet (2011) is a superhero action...
4        The Lord of the Rings: The Return of the Kin...
                             ...                        
995      "The Pirates of the Caribbean: Dead Man's Ch...
996     Martin Scorsese's The Wolf of Wall Street is ...
997     "The Lion King" is a stunning, visually-arres...
998     The Wiz, a 1975 musical fantasy film adaptati...
999     "The Bourne Identity" is a thought-provoking ...
Name: text, Length: 1000, dtype: object

### Removal of noise


In [4]:
def remove_noise(x):
    """method used for removal of instances that contain mostly repeating tokens ex. "MS MS MS MS ..."

    Args:
        x (str): text to check

    Returns:
        str: input text if tokens are not repeating else None
    """
    k, v = np.unique(x.split(), return_counts=True)
    if np.max(v)>len(k):
        print(x)
        return None
    return x

movies = movies.apply(lambda x:remove_noise(x))
movies = list(movies[~movies.isna()])

                                                                                                                                                                                                                                                                                                                                                                                                                                                                       MS   MSMS MS  MS MS  MSMS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS MS 

In [5]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, evidences=None):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.evidences=evidences

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
#             token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])   

In [16]:
i = 0
random_seed = 123

np.random.seed(random_seed)
random.seed(random_seed)
for i in range(5):
    # Loading imbalanced datasets
    with open(rf"C:\Users\Maciek\Documents\Studia\Magisterka\GitHub\BalancingMethodsNLP\nlpaug\eraser_movie_50_imbalanced_{i}.obj", 'rb') as pickle_file:
        imbalanced_dataset = pickle.load(pickle_file)
    
    # Combining imbalanced data and LLama generated revies
    generation_count = len(imbalanced_dataset.targets) - 2*np.sum(imbalanced_dataset.targets)
    balanced_inputs = imbalanced_dataset.inputs + random.sample(movies, generation_count)
    balanced_targets = imbalanced_dataset.targets + [1 for _ in range(generation_count)]

    # Shuffling inputs and targets after balancing
    np.random.seed(random_seed)
    np.random.shuffle(balanced_targets)
    np.random.seed(random_seed)
    np.random.shuffle(balanced_inputs)

    # Creating the dataset
    dataset = TrainerDataset(balanced_inputs, balanced_targets, imbalanced_dataset.tokenizer)
    
    # Saving created dataset to file
    filehandler = open(f'nlpaug/erqaser_movie_50_LLama_{i}.obj',"wb")
    pickle.dump(dataset,filehandler)
    filehandler.close()
    
    _, counts = np.unique(dataset.targets, return_counts=True)
    ratio = counts[0]/np.sum(counts)
    print(f"Ratio: {ratio}")
    print("Some examples from positive class")
    print(np.sort((np.array(dataset.inputs)[np.array(dataset.targets)==1]))[-3:])
    print("========================================================================================================================================================")
    print()
    print()

Ratio: 0.5
Some examples from positive class
["written by john grisham and robert altman ( as al hayes ) directed by robert altman some time ago , a young director was given the opportunity to direct the screen version of a popular novel .the novel was a summer smash , topping best - seller lists across the country and already drawing speculation as to which popular actors of the day might play the leads .the only problem : the novel was n't very good .it was far more pulp than pulitzer .but the director realized that , no matter how pedestrian the prose , this book really grabbed people - that the author , while not a great writer , was a fantastic storyteller .so he took on the project , convinced he could make something happen on screen .the director ?francis ford coppola .the book , of course , was mario puzo 's the godfather .the rest , as they say in showbiz , was history .by bringing up the godfather , i do n't mean to suggest that robert altman 's the gingerbread man is destine